# Cours LLM 
# AIFOUTE Othmane


# RNN

Lien du dataset: https://github.com/VincentChen95/Machine-Translation-Based-On-RNN-Model/tree/master/data

In [26]:
import collections
import numpy as np
import pickle
import nltk
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Embedding, SimpleRNN, GRU, LSTM, Bidirectional,
    Input, TimeDistributed, Dense
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

# Import custom helper module
import helper


In [27]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


In [28]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


In [29]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [30]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

In [31]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length == None:
        length = max([len(sentence) for sentence in x])
    print(length)
    return pad_sequences(x,maxlen=length,padding='post')


### Preprocess Pipeline

In [32]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

15
21
Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [33]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [34]:
def adjust_labels(y, vocab_size):
   
    y_adjusted = np.array(y)
    y_adjusted[y_adjusted >= vocab_size] = vocab_size - 1  
    return y_adjusted

preproc_french_sentences = adjust_labels(preproc_french_sentences, french_vocab_size)


## Metrique BLEU

In [35]:

def calculate_bleu(model, x_test, y_test, tokenizer, max_output_length=30):
    """
    Calcule le BLEU score sur un jeu de test
    :param model: Le modèle entraîné
    :param x_test: Les données d'entrée du jeu de test
    :param y_test: Les véritables phrases (références)
    :param tokenizer: Le tokenizer pour convertir les indices en mots
    :param max_output_length: Longueur maximale de la séquence de sortie
    :return: BLEU score moyen
    """
    total_bleu = 0
    num_samples = len(x_test)
    
    for i in range(num_samples):
        # Prédire la séquence pour une phrase en entrée
        pred = model.predict(x_test[i:i+1])  # Prediction for one sample at a time
        
        # Convertir les logits en indices de mots (utilisation de np.argmax pour obtenir les indices)
        pred_indices = np.argmax(pred[0], axis=-1)
        
        # Convertir les indices en mots à l'aide du tokenizer
        pred_sentence = [tokenizer.index_word.get(idx, '<UNK>') for idx in pred_indices]
        pred_sentence = ' '.join(pred_sentence).split('<PAD>')[0]  # Enlever le padding

        # Convertir les véritables indices en mots (références)
        true_sentence = [tokenizer.index_word.get(int(idx), '<UNK>') for idx in y_test[i]]  # Convertir chaque élément en int
        true_sentence = ' '.join(true_sentence).split('<PAD>')[0]  # Enlever le padding
        
        # Calculer le BLEU score pour cette prédiction
        bleu_score = sentence_bleu([true_sentence.split()], pred_sentence.split())
        total_bleu += bleu_score
        
    # Calculer le BLEU score moyen
    avg_bleu = total_bleu / num_samples
    return avg_bleu





In [52]:


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    #Config Hyperparameters
    learning_rate = 0.01
    
    #Config Model
    inputs = Input(shape=input_shape[1:])
    hidden_layer = SimpleRNN(output_sequence_length, return_sequences=True)(inputs)
    # The output is the french_vocab_size~
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(hidden_layer)
    #Create Model from parameters defined above
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

x_train, x_test, y_train, y_test = train_test_split(tmp_x, preproc_french_sentences, test_size=0.2, random_state=42)

# Train the neural network
simple_rnn_model = simple_model(
        x_train.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(x_train, y_train, batch_size=1024, epochs=50, validation_split=0.2)
simple_rnn_model.summary()
print(logits_to_text(simple_rnn_model.predict(x_test[:1])[0], french_tokenizer))


Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 24s 236ms/step - accuracy: 0.3208 - loss: 3.7272 - val_accuracy: 0.4512 - val_loss: 2.4589
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 20s 228ms/step - accuracy: 0.4710 - loss: 2.4034 - val_accuracy: 0.5135 - val_loss: 2.2613
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 22s 257ms/step - accuracy: 0.5122 - loss: 2.2396 - val_accuracy: 0.5251 - val_loss: 2.1576
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 20s 232ms/step - accuracy: 0.5270 - loss: 2.1196 - val_accuracy: 0.5300 - val_loss: 2.0227
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 21s 246ms/step - accuracy: 0.5295 - loss: 2.0077 - val_accuracy: 0.5321 - val_loss: 1.9588
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 22s 257ms/step - accuracy: 0.5308 - loss: 1.9464 - val_accuracy: 0.5310 - val_loss: 1.9183
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 24s 270ms/step - accuracy: 0.5345 - loss: 1.9080 - val_accuracy: 0.5363 - val_loss: 1.8894
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 25s 285ms/step - accuracy: 0.5394 - loss: 1.8741 - val_accu

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 21, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_9 (SimpleRNN)        │ (None, 21, 21)         │           483 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, 21, 344)        │         7,568 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,155 (94.36 KB)

 Trainable params: 8,051 (31.45 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 16,104 (62.91 KB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step
californie est est est en en et il est il est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [54]:
modelnamefile = 'rnn.sav'
pickle.dump(simple_rnn_model, open(modelnamefile, 'wb'))

In [59]:
#x_train, x_test, y_train, y_test = train_test_split(tmp_x, preproc_french_sentences, test_size=0.2, random_state=42)

# Sélectionner les 500 dernières lignes pour l'évaluation BLEU
bleu_score = calculate_bleu(simple_rnn_model, tmp_x[-500:], preproc_french_sentences[-500:], french_tokenizer)
print(f'BLEU Score: {bleu_score}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


C:\Users\othmane aifoute\AppData\Local\Temp\ipykernel_20252\3988699162.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  true_sentence = [tokenizer.index_word.get(int(idx), '<UNK>') for idx in y_test[i]]  # Convertir chaque élément en int


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━

c:\Users\othmane aifoute\AppData\Local\Programs\Python\Python312\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━

---------------

# LSTM

In [60]:


def lstm_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an LSTM-based model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Config Hyperparameters
    learning_rate = 0.001
    
    # Config Model
    inputs = Input(shape=input_shape[1:])
    # Remplacer SimpleRNN par LSTM
    hidden_layer = LSTM(output_sequence_length, return_sequences=True)(inputs)
    # The output is the french_vocab_size
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(hidden_layer)
    # Create Model from parameters defined above
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


# Reshaping the input to work with an LSTM
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

x_train, x_test, y_train, y_test = train_test_split(tmp_x, preproc_french_sentences, test_size=0.2, random_state=42)

# Train the neural network with LSTM model
lstm_model_instance = lstm_model(
        x_train.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

# Train the model
lstm_model_instance.fit(x_train, y_train, batch_size=1024, epochs=50, validation_split=0.2)
lstm_model_instance.summary()

# Test the model and print the result
print(logits_to_text(lstm_model_instance.predict(x_test[:1])[0], french_tokenizer))


Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 27s 268ms/step - accuracy: 0.2028 - loss: 5.4386 - val_accuracy: 0.4089 - val_loss: 3.7371
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 21s 246ms/step - accuracy: 0.4079 - loss: 3.3938 - val_accuracy: 0.4093 - val_loss: 2.8868
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 22s 254ms/step - accuracy: 0.4078 - loss: 2.8411 - val_accuracy: 0.4096 - val_loss: 2.7274
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 22s 252ms/step - accuracy: 0.4103 - loss: 2.7034 - val_accuracy: 0.4292 - val_loss: 2.6249
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 23s 264ms/step - accuracy: 0.4356 - loss: 2.6040 - val_accuracy: 0.4740 - val_loss: 2.5308
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 23s 259ms/step - accuracy: 0.4768 - loss: 2.5114 - val_accuracy: 0.4858 - val_loss: 2.4510
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 24s 274ms/step - accuracy: 0.4854 - loss: 2.4334 - val_accuracy: 0.4855 - val_loss: 2.3714
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 23s 260ms/step - accuracy: 0.4846 - loss: 2.3541 - val_accu

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 21, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 21, 21)         │         1,932 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_9              │ (None, 21, 344)        │         7,568 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,502 (111.34 KB)

 Trainable params: 9,500 (37.11 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 19,002 (74.23 KB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step
la est est parfois en en et il est est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [61]:
modelnamefile = 'lstm.sav'
pickle.dump(lstm_model_instance, open(modelnamefile, 'wb'))

In [62]:
#x_train, x_test, y_train, y_test = train_test_split(tmp_x, preproc_french_sentences, test_size=0.2, random_state=42)

# Sélectionner les 500 dernières lignes pour l'évaluation BLEU
bleu_score = calculate_bleu(lstm_model_instance, tmp_x[-500:], preproc_french_sentences[-500:], french_tokenizer)
print(f'BLEU Score: {bleu_score}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


C:\Users\othmane aifoute\AppData\Local\Temp\ipykernel_20252\3988699162.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  true_sentence = [tokenizer.index_word.get(int(idx), '<UNK>') for idx in y_test[i]]  # Convertir chaque élément en int


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━

----

# BILSTM

In [65]:

def simple_bilstm_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic BiLSTM model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Config Hyperparameters
    learning_rate = 0.001
    
    # Config Model
    inputs = Input(shape=input_shape[1:])
    hidden_layer = Bidirectional(LSTM(output_sequence_length, return_sequences=True))(inputs)
    # The output is the french_vocab_size
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(hidden_layer)
    
    # Create Model from parameters defined above
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


# Reshaping the input to work with a BiLSTM
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

x_train, x_test, y_train, y_test = train_test_split(tmp_x, preproc_french_sentences, test_size=0.2, random_state=42)


# Train the neural network with BiLSTM
simple_bilstm_model = simple_bilstm_model(
        x_train.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_bilstm_model.fit(x_train, y_train, batch_size=1024, epochs=50, validation_split=0.2)

simple_bilstm_model.summary()



Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 29s 285ms/step - accuracy: 0.1836 - loss: 5.4870 - val_accuracy: 0.4528 - val_loss: 3.3523
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 24s 272ms/step - accuracy: 0.4667 - loss: 2.9606 - val_accuracy: 0.4856 - val_loss: 2.5038
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 23s 270ms/step - accuracy: 0.4840 - loss: 2.4528 - val_accuracy: 0.4949 - val_loss: 2.3142
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 24s 271ms/step - accuracy: 0.4941 - loss: 2.2774 - val_accuracy: 0.5046 - val_loss: 2.1532
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 24s 273ms/step - accuracy: 0.5102 - loss: 2.1204 - val_accuracy: 0.5311 - val_loss: 2.0160
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 24s 276ms/step - accuracy: 0.5354 - loss: 1.9867 - val_accuracy: 0.5457 - val_loss: 1.9077
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 24s 279ms/step - accuracy: 0.5473 - loss: 1.8872 - val_accuracy: 0.5636 - val_loss: 1.8217
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 24s 277ms/step - accuracy: 0.5678 - loss: 1.8015 - val_accu

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, 21, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 21, 42)         │         3,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_11             │ (None, 21, 344)        │        14,792 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,970 (218.64 KB)

 Trainable params: 18,656 (72.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 37,314 (145.76 KB)

In [66]:
modelnamefile = 'bilstm.sav'
pickle.dump(simple_bilstm_model, open(modelnamefile, 'wb'))

In [67]:

# Apply BLEU score on the 500 last sentences
bleu_score = calculate_bleu(simple_bilstm_model, tmp_x[-500:], preproc_french_sentences[-500:], french_tokenizer)
print(f'BLEU Score: {bleu_score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 675ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


C:\Users\othmane aifoute\AppData\Local\Temp\ipykernel_20252\3988699162.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  true_sentence = [tokenizer.index_word.get(int(idx), '<UNK>') for idx in y_test[i]]  # Convertir chaque élément en int


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━

----

# GRU

In [36]:
def gru_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN with GRU on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Config Hyperparameters
    learning_rate = 0.01
    
    # Config Model
    inputs = Input(shape=input_shape[1:])
    hidden_layer = GRU(output_sequence_length, return_sequences=True)(inputs)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(hidden_layer)
    
    # Create Model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Example usage:
# Reshaping the input to work with a basic RNN
tmp_x = pad_sequences(preproc_english_sentences, maxlen=max_french_sequence_length, padding='post')
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

x_train, x_test, y_train, y_test = train_test_split(tmp_x, preproc_french_sentences, test_size=0.2, random_state=42)

# Train the neural network
gru_model = gru_model(
    x_train.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size
)
gru_model.fit(x_train, y_train, batch_size=1024, epochs=50, validation_split=0.2)
gru_model.summary()
print(logits_to_text(gru_model.predict(x_test[:1])[0], french_tokenizer))

Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 34s 289ms/step - accuracy: 0.4076 - loss: 3.7276 - val_accuracy: 0.4876 - val_loss: 2.3737
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 21s 242ms/step - accuracy: 0.4792 - loss: 2.3273 - val_accuracy: 0.4862 - val_loss: 2.1941
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 21s 243ms/step - accuracy: 0.4991 - loss: 2.1357 - val_accuracy: 0.5452 - val_loss: 1.9463
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 23s 266ms/step - accuracy: 0.5484 - loss: 1.8989 - val_accuracy: 0.5686 - val_loss: 1.7802
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 22s 247ms/step - accuracy: 0.5717 - loss: 1.7430 - val_accuracy: 0.5835 - val_loss: 1.6493
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 21s 245ms/step - accuracy: 0.5856 - loss: 1.6231 - val_accuracy: 0.5887 - val_loss: 1.5763
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 23s 263ms/step - accuracy: 0.5908 - loss: 1.5621 - val_accuracy: 0.5924 - val_loss: 1.5367
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 22s 254ms/step - accuracy: 0.5951 - loss: 1.5223 - val_accu

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 21, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 21, 21)         │         1,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 21, 344)        │         7,568 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,242 (106.42 KB)

 Trainable params: 9,080 (35.47 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 18,162 (70.95 KB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 509ms/step
californie est parfois parfois en l' et il il parfois parfois parfois en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [13]:
modelnamefile = 'gru.sav'
pickle.dump(gru_model, open(modelnamefile, 'wb'))

In [14]:

# Apply BLEU score on the 500 last sentences
bleu_score = calculate_bleu(gru_model, tmp_x[-500:], preproc_french_sentences[-500:], french_tokenizer)
print(f'BLEU Score: {bleu_score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step

C:\Users\othmane aifoute\AppData\Local\Temp\ipykernel_4924\3988699162.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  true_sentence = [tokenizer.index_word.get(int(idx), '<UNK>') for idx in y_test[i]]  # Convertir chaque élément en int


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━

-----

# BERT

In [1]:
from transformers import CamembertModel, CamembertTokenizer
from transformers import pipeline 

In [2]:
tokenizer = CamembertTokenizer.from_pretrained("camembert/camembert-base-wikipedia-4gb")
camembert = CamembertModel.from_pretrained("camembert/camembert-base-wikipedia-4gb")


sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

c:\Users\othmane aifoute\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\othmane aifoute\.cache\huggingface\hub\models--camembert--camembert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/509 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [4]:
camembert_fill_mask  = pipeline("fill-mask", model="camembert/camembert-base-wikipedia-4gb", tokenizer="camembert/camembert-base-wikipedia-4gb")
results = camembert_fill_mask("Le camembert est un fromage de <mask>!")

config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

c:\Users\othmane aifoute\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\othmane aifoute\.cache\huggingface\hub\models--camembert--camembert-base-wikipedia-4gb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

In [8]:
phrases = ["La france est située en <mask>!", "J'adore manger des <mask>", "Le chocolat est issu de la fêve de <mask>!", 
           "Emmanuel <mask> est le président de la France"]

result = []
for phrase in phrases: 
    result.append(camembert_fill_mask(phrase))

In [14]:
print(result)

[[{'score': 0.06966320425271988, 'token': 845, 'token_str': 'mer', 'sequence': 'La france est située en mer !'}, {'score': 0.047582078725099564, 'token': 1107, 'token_str': 'terre', 'sequence': 'La france est située en terre !'}, {'score': 0.03956795111298561, 'token': 1114, 'token_str': '!', 'sequence': 'La france est située en ! !'}, {'score': 0.038430895656347275, 'token': 90, 'token_str': 'France', 'sequence': 'La france est située en France !'}, {'score': 0.02393495850265026, 'token': 2301, 'token_str': 'forêt', 'sequence': 'La france est située en forêt !'}], [{'score': 0.0340404212474823, 'token': 17431, 'token_str': 'champignon', 'sequence': "J'adore manger des champignon"}, {'score': 0.02699596807360649, 'token': 7260, 'token_str': 'poissons', 'sequence': "J'adore manger des poissons"}, {'score': 0.025112522765994072, 'token': 18401, 'token_str': 'champignons', 'sequence': "J'adore manger des champignons"}, {'score': 0.024569997563958168, 'token': 9254, 'token_str': 'œufs', 's